In [2]:
# importing libraries
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import GEOparse
import plotly.graph_objects as go
from scipy.stats import pearsonr, spearmanr, kendalltau

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [3]:
class GEOParser:
    """
    Parser for GEO datasets using GEOparse.
    First version specific for GSE121239 (Lúpus).

    Parameters
    -----------
    geo_id : str
        GEO Series ID (e.g., 'GSE121239').
    destdir : str
        Directory to download GEO data.
    gene_id_cols : list of str, optional
        Columns to use for gene ID mapping. Default is ['ID', 'Gene Symbol', 'Gene Title'].
    meta_map : dict, optional
        Mapping of metadata fields to their index in characteristics_ch1. 
    """
    def __init__(self, geo_id, destdir, gene_id_cols=None, meta_map=None):
        self.geo_id = geo_id
        self.destdir = destdir
        self.gene_id_cols = gene_id_cols or ['ID', 'Gene Symbol', 'Gene Title']
        self.meta_map = meta_map
        self.gse = None
        self.gpl = None

    def load(self):
        """
        Load GEO Series data.
        """
        self.gse = GEOparse.get_GEO(geo=self.geo_id, destdir=self.destdir, silent=True)

    def try_cast(self, val, typ):
        """
        Try to cast a value to a specified type, return NaN on failure.

        Parameters
        ----------
        val : any
            Value to cast.
        typ : type
            Target type for casting.
        """
        try:
            return typ(val)
        except (ValueError, TypeError):
            return np.nan

    def select_main_gene(self, gene_str, separator='///'):
        """
        Select the main gene from a string of gene symbols separated by a given separator.
        Parameters
        ----------
        gene_str : str
            String containing gene symbols separated by the specified separator.
        separator : str, optional
            Separator used in the gene string. Default is '///'.
        Returns
        -------
        main_gene : str or np.nan
            The selected main gene symbol, or NaN if input is NaN.
        """
        if pd.isna(gene_str):
            return np.nan
        # Lista de genes encontrados
        genes = [g.strip() for g in gene_str.split(separator)]

        # Prioriza genes que não começam com LOC (Locus genômico)
        known_genes = [g for g in genes if not g.startswith('LOC')]
        if known_genes:
            return known_genes[0]
        return genes[0]  # Se só tem LOC, retorna o primeiro

    def parse(self):
        """
        Parse the loaded GEO Series data.

        Returns
        -------
        gene_data: pd.DataFrame
            Dataframe with Gene expression data.
        patient_data: pd.DataFrame
            Dataframe with Patient metadata.
        """
        if self.gse is None:
            self.load()

        # 1. Cria listas com metadados e expressao dos paciente e expressao genica 
        patient_meta_list = []
        gene_data_list = []

        # 2. Itera sobre os GSMs para extrair metadados e dados de expressão
        for gsm_name, gsm in self.gse.gsms.items():

            # 2.1. Extrai metadados do paciente
            meta = gsm.metadata['characteristics_ch1']
            patient_meta = {'sample_code': gsm_name, 'treatment_protocol': gsm.metadata['treatment_protocol_ch1'][0]}

            # 2.2. Parse de metadados de acordo com o mapeamento fornecido
            for key, idx in self.meta_map.items():
                # 2.3 . Extrai valor da lista de metadados e faz cast apropriado
                val = meta[idx].split(':')[1].strip() if len(meta) > idx else None
                if key == 'sledai':
                    val = self.try_cast(val, int)
                elif key == 'visit_date':
                    val = self.try_cast(val, pd.Timestamp)
                elif key == 'neutrophil_percent':
                    val = self.try_cast(val, float)
                # 2.4. Adiciona informação ao dicionario de metadados do paciente
                patient_meta[key] = val
            
            patient_meta_list.append(patient_meta)

            # 2.5. Extrai dados de expressão gênica
            gene_df = gsm.table[['ID_REF', 'VALUE']].copy()
            gene_df['sample_code'] = gsm_name
            gene_data_list.append(gene_df)

        # 3. Concatena dados de expressão gênica em um único DataFrame
        gene_data = pd.concat(gene_data_list, ignore_index=True)

        # 4. GPLS: plataforma com anotação de genes, incluindo nome e símbolo
        self.gpl = self.gse.gpls[list(self.gse.gpls.keys())[0]].table

        # 5. Recupera nome e título do gene com base no ID
        gene_data = gene_data.merge(self.gpl[self.gene_id_cols], left_on='ID_REF', right_on='ID', how='left')
        gene_data = gene_data.rename(columns={'VALUE': 'gene_expression_value', 'Gene Symbol': 'gene_symbol', 'Gene Title': 'gene_title'})
        gene_data = gene_data[['sample_code', 'ID', 'gene_symbol', 'gene_title', 'gene_expression_value']]

        # 6. Metadados do paciente em dataframe
        patient_data = pd.DataFrame(patient_meta_list)
        patient_data['patient_id'] = patient_data['patient_id'].replace({'NA': np.nan})
        patient_data = patient_data.sort_values(by=['patient_id', 'visit_date'])

        return gene_data, patient_data

### Loading Data

In [ ]:
meta_map = {
    'state': 0,
    'patient_id': 1,
    'sledai': 2,
    'visit_date': 3,
    'neutrophil_percent': 5
}

# Cria objeto parser
parser = GEOParser(geo_id="GSE121239", destdir="../../data/raw/", meta_map=meta_map)

# Parse dos dados e armazena em tabelas
gene_data, patient_data = parser.parse()

# Mantem o "gene principal" em casos de genes separados por '///'
gene_data['main_gene'] = gene_data['gene_symbol'].apply(parser.select_main_gene)

# Remove valores de microarray (Affymetrix) -> Controle de qualidade de experimento
gene_data = gene_data[~gene_data['ID'].str.startswith('AFFX')]

/home/danbrz/Projects/project/.venv/lib/python3.12/site-packages/GEOparse/GEOparse.py:401: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  return read_csv(StringIO(data), index_col=None, sep="\t")


In [ ]:
patient_data = patient_data.sort_values(by=['patient_id', 'visit_date'])
patient_data['sledai_variation'] = patient_data.groupby('patient_id')['sledai'].diff()
patient_data['visit_date_diff'] = patient_data.groupby('patient_id')['visit_date'].diff()

# 70/227 (31%) tiveram aumento de SLEDAI entre visitas
# 157/227 (69%) tiveram diminuição ou estabilidade de SLEDAI entre visitas
# Não existe padrão de tempo entre visitas (provavelmente não era programada)

In [ ]:
patient_data.head()

,sample_code,treatment_protocol,state,patient_id,sledai,visit_date,neutrophil_percent,sledai_variation,visit_date_diff
105,GSM3428415,Standard.,Systemic Lupus Erythematosus,1001,4,2009-10-08,65.2,NaN,NaT
106,GSM3428416,Standard.,Systemic Lupus Erythematosus,1001,2,2010-01-11,61.5,-2.0,95 days
107,GSM3428417,Standard.,Systemic Lupus Erythematosus,1001,6,2010-03-29,57.5,4.0,77 days
108,GSM3428418,Standard.,Systemic Lupus Erythematosus,1041,2,2009-09-24,73.8,NaN,NaT
109,GSM3428419,Standard.,Systemic Lupus Erythematosus,1041,10,2009-12-10,64.1,8.0,77 days


In [ ]:
gene_data.head()

,sample_code,ID,gene_symbol,gene_title,gene_expression_value,main_gene
0,GSM3428310,1007_PM_s_at,DDR1,discoidin domain receptor tyrosine kinase 1,4.955257,DDR1
1,GSM3428310,1053_PM_at,RFC2,"replication factor C (activator 1) 2, 40kDa",5.984784,RFC2
2,GSM3428310,117_PM_at,HSPA6,heat shock 70kDa protein 6 (HSP70B'),9.477945,HSPA6
3,GSM3428310,121_PM_at,PAX8,paired box 8,4.553229,PAX8
4,GSM3428310,1255_PM_g_at,GUCA1A,guanylate cyclase activator 1A (retina),1.921190,GUCA1A


In [ ]:
merged = gene_data.merge(right=patient_data, on = 'sample_code').dropna(subset='neutrophil_percent')
merged.head()

,sample_code,ID,gene_symbol,gene_title,gene_expression_value,main_gene,treatment_protocol,state,patient_id,sledai,visit_date,neutrophil_percent,sledai_variation,visit_date_diff
1092260,GSM3428330,1007_PM_s_at,DDR1,discoidin domain receptor tyrosine kinase 1,4.231006,DDR1,Standard.,Systemic Lupus Erythematosus,24,4,2009-10-15,89.3,NaN,NaT
1092261,GSM3428330,1053_PM_at,RFC2,"replication factor C (activator 1) 2, 40kDa",6.468179,RFC2,Standard.,Systemic Lupus Erythematosus,24,4,2009-10-15,89.3,NaN,NaT
1092262,GSM3428330,117_PM_at,HSPA6,heat shock 70kDa protein 6 (HSP70B'),10.030950,HSPA6,Standard.,Systemic Lupus Erythematosus,24,4,2009-10-15,89.3,NaN,NaT
1092263,GSM3428330,121_PM_at,PAX8,paired box 8,4.773703,PAX8,Standard.,Systemic Lupus Erythematosus,24,4,2009-10-15,89.3,NaN,NaT
1092264,GSM3428330,1255_PM_g_at,GUCA1A,guanylate cyclase activator 1A (retina),1.859223,GUCA1A,Standard.,Systemic Lupus Erythematosus,24,4,2009-10-15,89.3,NaN,NaT


In [ ]:
tgt_genes=     [
    "RPL4",
    "RPLP0",
    "EEF2",
    "RPL12",
    "KIAA0649", # "PPP1R26",
    "EEF1G",
    "RPS17",
    "KARS"#"KARS1"
]
df_corr = pd.DataFrame(index=[ 'pearsonr', 'spearmanr', 'kendalltau'])
for gene in tgt_genes:
    x = merged[merged['main_gene'] == gene][['gene_expression_value','neutrophil_percent']].to_numpy()
    if len(x)>1:
        exp = x[:,0]
        neut = x[:,1]

        fig = go.Figure(data=go.Scatter(x=exp, y=neut, mode='markers'))
        
        fig.update_layout(
            title=gene + ' - ' +'Correlação Expressão gênica x Porcentagem de neutrófilos',
            xaxis_title='Expressão gênica',
            yaxis_title='Porcentagem de neutrófilos',
            title_x=0.5  # centraliza o título
        )
        
        fig.show()
        fig.write_image(f"corr_{gene}.png", scale=2)  
        fig.show()
        corr_p,_ = pearsonr(exp,neut)
        print('Correlação de pearson: ', corr_p)
        corr_s,_ = spearmanr(exp,neut)
        print('Correlação de spearman: ', corr_s)
        corr_k,_ = kendalltau(exp,neut)
        print('Correlação de kendall: ', corr_k)
        df_corr[gene] = [corr_p,corr_s, corr_k]

df_corr.head()

Correlação de pearson:  -0.5587066496889658
Correlação de spearman:  -0.578481872524445
Correlação de kendall:  -0.41053841694504517


Correlação de pearson:  -0.4308145307927245
Correlação de spearman:  -0.5040973945390925
Correlação de kendall:  -0.37135733063475


Correlação de pearson:  -0.30285746412189646
Correlação de spearman:  -0.32299273989082533
Correlação de kendall:  -0.22031255979913494


Correlação de pearson:  -0.4516966542259414
Correlação de spearman:  -0.4628404737901759
Correlação de kendall:  -0.3160269008706994


Correlação de pearson:  -0.22644891877404372
Correlação de spearman:  -0.2062829478526679
Correlação de kendall:  -0.1379430154639939


Correlação de pearson:  -0.5837807947718944
Correlação de spearman:  -0.6038414933235983
Correlação de kendall:  -0.42938159890398103


Correlação de pearson:  -0.6471214507559517
Correlação de spearman:  -0.6736218465984876
Correlação de kendall:  -0.49027577533308087


Correlação de pearson:  -0.2372041618197872
Correlação de spearman:  -0.22873991213497705
Correlação de kendall:  -0.1600552467506249


,RPL4,RPLP0,EEF2,RPL12,KIAA0649,EEF1G,RPS17,KARS
pearsonr,-0.558707,-0.430815,-0.302857,-0.451697,-0.226449,-0.583781,-0.647121,-0.237204
spearmanr,-0.578482,-0.504097,-0.322993,-0.462840,-0.206283,-0.603841,-0.673622,-0.228740
kendalltau,-0.410538,-0.371357,-0.220313,-0.316027,-0.137943,-0.429382,-0.490276,-0.160055


In [ ]:
(df_corr.T).drop(columns='kendalltau').round(2).sort_values('pearsonr')
    # "RPL4",
    # "RPLP0",
    # "EEF2",
    # "RPL12",
    # "KIAA0649", # "PPP1R26",
    # "EEF1G",
    # "RPS17",
    # "KARS"#"KARS1"

NameError: name 'df_corr' is not defined